<a href="https://colab.research.google.com/github/renaldoaluska/pbagasal2025-klp11-garuda/blob/main/Scrapping%20Garuda%20Indonesia%20News%20Article/Scrapping%20Garuda%20Indonesia%20Article%20Link%20News/Scraping_GarudaIndonesia_LinkNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scraping Garuda Indonesia Link News**

Scraping link dilakukan pada berita yang berhubungan dengan Garuda Indonesia menggunakan `Selenium` pada Google Chrome.

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,014 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pa

In [2]:
# Install Libraries
!pip install chromedriver-autoinstaller
!pip install dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 21.0 MB/s eta 0:00:00


In [3]:
# Import library
import re
import csv
import sys
import time
import dateparser
import pandas as pd
import chromedriver_autoinstaller
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# Konfigurasi opsi Chrome
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Agar berjalan di mode tanpa antarmuka
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'/usr/local/lib/python3.12/dist-packages/chromedriver_autoinstaller/140/chromedriver'

# Scraping Link Artikel Berita di Google

In [5]:
# Inisialisasi list untuk menyimpan semua data
all_articles = set()

In [6]:
# Fungsi untuk memodifikasi link Google News
def rearrange_news_link(url):
  # Memisahkan URL berdasarkan pola 'start=' diikuti angka dan '&sa='
  url_split = re.split(r"start=(\d+)&sa=", url)
  # Mengembalikan URL yang telah diubah
  return url_split[0] + 'start=', '&sa=' + url_split[-1]

In [7]:
# Fungsi untuk hanya mengambil bagian utama dari link berita yang didapatkan
def get_google_main_url(url):
  return re.findall(r"^(.*?)(?=&)", url)[0]

In [26]:
def scrape_google_news_link(result_set, number, url, hl='en', gl='US'):
    # Memisahkan URL menjadi bagian yang bisa diproses
    url1, url2 = rearrange_news_link(url)

    # Menambahkan parameter bahasa dan lokasi ke URL
    url1 = f"{url1}hl={hl}&gl={gl}&"


    # Mendapatkan URL utama dari URL yang telah dipisahkan
    main_url = get_google_main_url(url1)
    print(f"Mengakses URL: {main_url}")

    # Menginisialisasi driver Chrome dengan opsi yang sudah ditentukan
    driver = webdriver.Chrome(options=chrome_options)
    article_xpath = "//div[contains(@class, 'MjjYud')]/div"

    try:
        # Mengambil artikel dari URL yang diberikan, sebanyak <number> artikel
        for i in range(0, number, 10):
            # Membuat URL untuk halaman artikel berikutnya
            page = url1 + str(i) + url2
            driver.get(page)
            time.sleep(2)  # Memberi waktu untuk halaman memuat

            # Menemukan elemen yang berisi artikel
            articles = driver.find_element(By.XPATH, article_xpath)
            articles = articles.find_elements(By.CLASS_NAME, "SoaBEf")
            print(f"Mengumpulkan artikel dari {i+1} hingga {i+10}")

            # Mengambil link, judul, dan tanggal dari setiap artikel
            for article in articles:
                # Mengambil link dari elemen <a>
                a = article.find_element(By.TAG_NAME, "a").get_attribute("href")

                 # Mengambil judul berita dari elemen dengan class "n0jPhd ynAwRc MBeuO nDgy9d"
                try:
                    title = article.find_element(By.XPATH, ".//div[@class='n0jPhd ynAwRc MBeuO nDgy9d']").text
                except NoSuchElementException:
                    title = None

                # Mengambil tanggal dari elemen dengan kelas 'OSrXXb'
                try:
                    date_str = article.find_element(By.CLASS_NAME, "OSrXXb").text
                    date = dateparser.parse(date_string=date_str, languages=['id'])
                    date = date.strftime('%Y-%m-%d') if date else None
                except NoSuchElementException:
                    date = None  # Mengisi dengan None jika tidak ditemukan

                # Mengambil nama portal dari elemen dengan class 'MgUUmf NUnG9d'
                try:
                    portal = article.find_element(By.XPATH, ".//div[@class='MgUUmf NUnG9d']").text
                except NoSuchElementException:
                    portal = None

                # Menambahkan tuple (link, judul, tanggal, portal, hashtag) ke hasil
                result_set.add(tuple([a, title, date, portal]))

    except NoSuchElementException as e:
        # Jika elemen artikel tidak ditemukan
        print('Halaman artikel sudah tidak tersedia')

    finally:
        # Menutup driver setelah proses selesai
        driver.close()
        print(f"Proses selesai. Total link artikel unik yang berhasil diambil: {len(result_set)}")
        print('----------------------------------------------------------')

In [27]:
# Link Google News yang akan di-scraping
# garuda indonesia
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=garuda+indonesia&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIIiJ31AXdt_1uPlcuRXCrWbOCJRPg:1725864313300&ei=eZneZv-IEoKgseMPiN7GyQs&start=0&sa=N&ved=2ahUKEwi_qczKobWIAxUCUGwGHQivMbk4ChDy0wN6BAgDEAQ&biw=1536&bih=703&dpr=1.25', hl='id', gl='ID')
# pesawat garuda
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=pesawat+garuda&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWII8LRnsJNcGclJCB8Yvnvf7td5KGQ:1725864448748&ei=AJreZra2Ld-R4-EP4YLriAM&start=0&sa=N&ved=2ahUKEwi2tpeLorWIAxXfyDgGHWHBGjE4ChDy0wN6BAgDEAQ&biw=1536&bih=703&dpr=1.25', hl='id', gl='ID')

Mengakses URL: https://www.google.com/search?q=garuda+indonesia
Halaman artikel sudah tidak tersedia
Proses selesai. Total link artikel unik yang berhasil diambil: 1194
----------------------------------------------------------
Mengakses URL: https://www.google.com/search?q=pesawat+garuda
Halaman artikel sudah tidak tersedia
Proses selesai. Total link artikel unik yang berhasil diambil: 1194
----------------------------------------------------------


In [29]:
# Contoh isi link_set
for i, data in enumerate(all_articles):
  if (i == 5):
    break
  print(data)

('https://www.travelandtourworld.com/news/article/garuda-indonesia-now-expands-flight-network-to-bali-yogyakarta-surabaya-and-other-main-tourism-hub-by-2029-heres-what-you-need-to-know/', 'Garuda Indonesia Now Expands Flight Network to Bali, Yogyakarta, Surabaya, and Other Main Tourism Hub by 2029: Here’s What You Need To Know', '2025-07-31', 'Travel And Tour World')
('https://www.ft.com/content/45ff1892-b4fe-4caf-9d07-2e6a0a189fed', 'Islamic bonds come under microscope after Garuda Indonesia default', '2021-08-17', 'Financial Times')
('https://ulasan.co/beredar-video-ban-pesawat-garuda-menggelinding-di-landasan-pacu-tanjungpinang/', 'Beredar Video Ban Pesawat Garuda Menggelinding di Landasan Pacu Tanjungpinang', '2025-04-16', 'Ulasan.co')
('https://www.cnnindonesia.com/ekonomi/20240824164025-92-1137068/135-penumpang-pesawat-tangki-bocor-garuda-sudah-diterbangkan-kembali', '135 Penumpang Pesawat Tangki Bocor Garuda Sudah Diterbangkan Kembali', '2024-08-24', 'CNN Indonesia')
('https://t

## Ubah Set Menjadi DataFrame

In [23]:
#newsLink_df = pd.DataFrame(data=all_articles, columns=['link', 'tanggal'])
newsLink_df = pd.DataFrame(data=all_articles, columns=['link', 'judul', 'tanggal', 'portal'])

In [24]:
# Konversi tanggal menjadi datetime, urutkan dari terbaru ke terlama
newsLink_df['tanggal'] = pd.to_datetime(newsLink_df['tanggal'], format='%Y-%m-%d')
newsLink_df = newsLink_df.sort_values(by='tanggal', ascending=False, ignore_index=True)

In [25]:
newsLink_df

,link,judul,tanggal,portal
0,https://kumparan.com/kumparanbisnis/garuda-ind...,Garuda Indonesia Kembali RUPSLB di Tengah Isu ...,2025-09-30,Kumparan
1,https://voi.id/en/economy/519004,Commission V DPR Will Investigate Allegations ...,2025-09-29,VOI.ID
2,https://www.prnewswire.com/news-releases/garud...,Garuda Indonesia Goes Digital: Air Cargo Capac...,2025-09-29,PR Newswire
3,https://in.investing.com/news/company-news/gar...,Garuda Indonesia adds air cargo capacity to We...,2025-09-29,Investing.com India
4,https://jambi.pikiran-rakyat.com/info-data/pr-...,Jadwal Kedatangan Pesawat di Bandara Sultan Th...,2025-09-29,Jambian
...,...,...,...,...
1189,https://www.thejakartapost.com/indonesia/2024/...,Garuda Indonesia flight makes emergency landin...,NaT,The Jakarta Post
1190,https://finance.detik.com/bursa-dan-valas/d-81...,"Gelar RUPSLB Lagi, Anak Usaha Garuda Tunjuk Di...",NaT,detikFinance
1191,https://djsaviation.net/garuda-indonesia-aircr...,Garuda Indonesia Aircraft Are Grounded,NaT,Dj's Aviation
1192,https://www.aviacionline.com/qatar-airways-and...,Qatar Airways and Garuda Indonesia expand part...,NaT,Aviacionline


## Simpan Link Artikel Berita ke CSV dan Excel

In [18]:
output_file = 'link_berita_garudaindonesia.csv'
newsLink_df.to_csv(output_file, index=False)

message = f"Successfully output the results to the {output_file} file"
print('-' * len(message))
print(message)
print(f"Successfully got {len(all_articles)} unique article links")

---------------------------------------------------------------------------
Successfully output the results to the link_berita_garudaindonesia.csv file
Successfully got 1194 unique article links


In [19]:
output_file2 = 'link_berita_garudaindonesia.xlsx'
newsLink_df.to_excel(output_file2, index=False)